# DLL Layer: ActualDLL and ShimDLL

This notebook simulates the **middleware layer** in a vehicle diagnostic architecture.

It contains two components:
- **ActualDLL**: Talks directly to the VDA and retrieves the real VIN.
- **ShimDLL**: Intercepts communication between the diagnostic application and ActualDLL, modifies the VIN, and returns a fake one.

We use threading to start both servers in the background so they can accept incoming TCP connections.

In [ ]:
import socket
import threading

try:
    import nbformat
except ImportError:
    print("Installing nbformat...")
    !pip install nbformat

# Load CAN-layer functions from another notebook
%run ./VDA.ipynb

## Configuration

We use two different ports:
- Port `1337` is where the diagnostic application always connects.
- Port `1555` is used by the ActualDLL only when the shim is present.

By toggling the `USE_PORT_FOR_SHIM_MODE` flag, we control whether the ActualDLL listens on the shim's backend port or directly on the application port.

In [ ]:
USE_PORT_FOR_SHIM_MODE = False  # Set to False to run without shim

PORT = 1555 if USE_PORT_FOR_SHIM_MODE else 1337

# This server simulates the actual DLL that talks to CAN
def actualdll_socket_server():
    tcp_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    tcp_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) 
    tcp_sock.bind(("localhost", PORT))  # Port depends on mode
    tcp_sock.listen(1)
    print(f"[ActualDLL] Listening on port {PORT}...")

    conn, _ = tcp_sock.accept()
    with conn:
        req = conn.recv(1024).decode()
        if req == "VIN_REQUEST":
            vin = get_vin_from_can()  # From vda.ipynb
            print(f"[ActualDLL] Real VIN: {vin}")
            conn.sendall(vin.encode())
    tcp_sock.close()

# Start the actual DLL server in the background
actual_dll_start = threading.Thread(target=actualdll_socket_server, daemon=True)
actual_dll_start.start()

In [ ]:
actual_dll_start.join()

## ShimDLL: Intercepting and Modifying the VIN

This component pretends to be the ActualDLL by listening on port `1337`, but internally forwards requests to the real ActualDLL (on port `1555`), modifies the result, and returns the fake VIN.

In [ ]:
# This server intercepts the client's connection and modifies the VIN
def shimdll_socket_server():
    tcp_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    tcp_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) 
    tcp_sock.bind(("localhost", 1337))  # Replaces actualdll on this port
    tcp_sock.listen(1)
    print("[ShimDLL] Intercepting requests on port 1337...")

    conn, _ = tcp_sock.accept()
    with conn:
        req = conn.recv(1024).decode()
        if req == "VIN_REQUEST":
            # Connect to actualdll behind the scenes
            forward_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            forward_sock.connect(("localhost", 1555))
            forward_sock.sendall(b"VIN_REQUEST")

            # Receive real VIN
            real_vin = forward_sock.recv(1024).decode()
            print(f"[ShimDLL] Real VIN: {real_vin}")

            # Modify VIN before sending it back
            modified_vin = real_vin[:10] + "HACKED123"
            print(f"[ShimDLL] Sending Modified VIN: {modified_vin}")
            conn.sendall(modified_vin.encode())

            forward_sock.close()
    tcp_sock.close()

# Start the shim DLL server in the background
shim_thread = threading.Thread(target=shimdll_socket_server)
shim_thread.start()

In [ ]:
shim_thread.join()

## ✅ Summary

You now have two background servers running:
- `ActualDLL`: provides the real VIN from the CAN bus.
- `ShimDLL`: modifies the VIN on the fly and sends it to the client.

> Now, go back to the `Diagnostic_Application.ipynb` and re-run the cell.  
> You'll see the **modified VIN** (e.g., ending in `HACKED123`) anything changed.

This demonstrates how **middleware attacks** can be used to tamper with diagnostic communication in vehicles.